# RedGrease Demo at RedisConf 2021

Quick demonstration of how to create and run Redis Gears functions, using RedGrease.

## [Demos](#Demos):
1. [The Basics](#1.-The-Basics)
2. [Simple Analytics Query](#2.-Simple-Analytics-Query)
3. [Transaction Stream Processing](#3.-Transaction-Stream-Processing)
4. [Custom Command](#4.-Custom-Command)


# Preparations
Before running the demos, make sure that the prerequisites are met and that the preparation steps have successfully been executed. 
Some preparation steps, particularly the downloads, may take quite some time. 

## 1. Prerequisites
- Python3.7
- Pip
- Docker
- Jupyter

Run the cell below tho validate your prerequisites.

In [ ]:
# Run cell to test your environment requirements

import sys
import re
pyver = !{sys.executable} --version  # type: ignore
pipver = !{sys.executable} -m pip --version  # type: ignore
dockver = !docker --version  # type: ignore

if not re.match("Python 3.7", pyver[0]):
    raise SystemExit(f"This demo only supports Python 3.7. You are running {pyver[0]}.")

if not re.match(".*\(python 3.7\)", pipver[0]):
    raise SystemExit("Please install Pip for yout Python 3.7 environment.")

if not re.match("Docker version", dockver[0]):
    raise SystemExit("Please install Docker")

print("Requirements all look good!")

## 2. Python Requirements

Install the Python packages required for the demo:

- `redgrease[client]` - The RedGrease client library for Redis Gears. This is what is being demonstrated.

- `ipywidgets` - Jupyter notebook exetension, for displaying widgets, e.g. buttons, in this notebook.
- `requests` - For downloading content.

Run the cell below to install the requirements.

In [ ]:
%%capture reqs_install_output
!{sys.executable} -m pip install redgrease[client] ipywidgets requests
!jupyter nbextension enable --py widgetsnbextension

## 3. Download Datasets
Some of the demos requiere a portion of the [COCO Dataset](https://cocodataset.org) to be uploaded into the Redis Gears Cluster.
The COCO Dataset (Common Objects in Context) is a fairly large set of (~247,000) images and corresponding annotations of what tey are depicting.

### Example:
<img src="coco_example.jpg" > [COCO Example](coco_example.jpg)

```
a man riding a snowboard down a ski slope.
a snowboarder sailing down a snowy hillside on a mountain.
a man is snowboarding past blue markers on a mountain.
a man on a snowboard in the snow.
a man snow boarding in the snow on a slope. 
```


For the demo we will only pre-download the annotations (json), not the images (jpeg), but it is still between 250 - 500 MB of data, depending on which portions you choose.

There are two annotation packages to choose from. 
- **COCO Train/Cal 2014** - Annotations for 124,000 images (241 MB)
- **COCO Train/Val 2017** - Annotations for 123,000 images (241 MB)

Either or both may be used. 
Run the cell below and select using the buttons which dataset(s) to download.

In [ ]:
# This code is just for preparation of the demo.
# It is NOT part of the demo itself
#
# Download COCO Annotations 
# Run the cell, then:
# - Validate or modify the Download directory
# - Click the button, or buttons for the annotations to download

import ipywidgets as widgets
import os
import requests

coco_annotations_url = "http://images.cocodataset.org/annotations"
annotations_file_pattern = "annotations_trainval{}.zip"

layout = widgets.Layout(width="30%")
output = widgets.Output()

def get_download_path():
    download_dir = "."
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            download_dir = winreg.QueryValueEx(key, downloads_guid)[0]
    else:
        download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

    return os.path.join(download_dir, "COCO")

download_location = widgets.Text(
    value=get_download_path(),
    placeholder="Download directory",
    description="Directory to download annotations to.",
    layout=layout,
)
display(download_location)

def dl_state(button, downloading=None):
    year = button.value
    annotations_file_name = annotations_file_pattern.format(year)
    destination = os.path.join(download_location.value, annotations_file_name)
    is_downloaded = os.path.isfile(destination)
    button.disabled = is_downloaded or downloading is not None
    if downloading:
        button.description=f"Downloading COCO {year} annotations (241 MB): {downloading}%. Please wait!"
    elif is_downloaded:
        button.description=f"Congrats! COCO {year} annotataions is downloaded!"
    else:
        button.description=f"Download COCO {year} annotations (241 MB)"
    return is_downloaded, annotations_file_name, destination


def download_button_pressed(btn):
    downloaded, file_name, destination = dl_state(btn)
    if downloaded:
        return
    if not os.path.isdir(download_location.value):
        os.mkdir(download_location.value)
    try:
        response = requests.get(
            f"{coco_annotations_url}/{file_name}",
            stream=True
        )
        total_length = response.headers.get('content-length')
        with open(destination, "wb") as f:
            if total_length is None: # no content length header
                dl_state(btn, "???")
                f.write(response.content)
                return
            total_length = int(total_length)
            dl = 0
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
                dl_state(btn, int(100*(dl/total_length)))

    except Exception:
        try:
            os.remove(destination)
        except Exception:
            pass
    finally:
        dl_state(btn)

for year in ["2014", "2017"]:
    download_button = widgets.Button(
        tooltip='Start download of selected datasets into the selected download directory.',
        layout=layout
    )
    download_button.value = year
    dl_state(download_button)
    download_button.on_click(download_button_pressed)
    display(download_button)

display(output)

## 4. Download and run Redis Gears Cluster Docker image
Run the cell below to download a Redis Gears Cluster Docker image (~605 MB), if not already present, and run it. 

In [ ]:
redis_gears_cluster_image = "redislabs/rgcluster:1.0.6"
redis_gears_cluster_container_name = "demo_gears_cluster"

redis_gears_single_image = "redislabs/redisgears:1.0.6"
redis_gears_single_container_name = "demo_gears_single"

# Get the correct Redis Gears Images
!docker pull {redis_gears_single_image}
!docker pull {redis_gears_cluster_image}

# Check if the single container is already running.
container_info = !docker container inspect {redis_gears_single_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears single instance")
    !docker run --name {redis_gears_single_container_name} --rm -d -p 6379:6379 {redis_gears_single_image}


# Check if the cluster container is already running.
container_info = !docker container inspect {redis_gears_cluster_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears cluster instance")
    !docker run --name {redis_gears_cluster_container_name} --rm -d -p 30001:30001 -p 30002:30002 -p 30003:30003 {redis_gears_cluster_image}

print("Redis Gears containers are running!")

## 5. Load Annotation Data into Redis cluster
By running the cell below, the COCO annotations downloaded above will be loaded into the Redis Cluster.

In [ ]:
import glob
import itertools
import json
import os
import re
import redgrease
import zipfile

annotation_archive_files = os.path.join(download_location.value, "annotations_trainval*.zip")
annotation_archives = glob.glob(annotation_archive_files)

if not annotation_archives:
    print("no archives")
    raise SystemExit("Please download either or both COCO annotations as per instructions above.")

r = redgrease.RedisGears(host="localhost", port=30001)

annotation_json_pattern = re.compile("annotations/(\w+)_([a-zA-Z]+)([0-9]+).json")

annotation_types = ["instances", "person_keypoints", "captions"]
years = ["2014", "2017"]
purpose = ["val", "train"]

output = widgets.Output()
progress = widgets.Text("", layout=layout)

def load_annotation_info(base_key, info):
    annotation_info_key = f"{base_key}/info"
    r.hset(annotation_info_key, mapping=info)
    return annotation_info_key

def load_license_info(base_key, license):
    license_key = f"{base_key}/license/{license['id']}"
    if not r.exists(license_key):
        r.hset(license_key, mapping=license)
    return license_key

def load_image_info(base_key, image_info):
    img_info_key = f"{base_key}/image/{image_info['id']}/info"
    if not r.exists(img_info_key):
        r.hset(img_info_key, mapping=image_info)
    return img_info_key

def load_keypoint_names(base_key, keypoints):
    keypoints_key = f"{base_key}/keypoints"
    r.lpush(keypoints_key, *keypoints)
    return keypoints_key

def load_list_of_str(base_key, sequence):
    list_key = f"{base_key}/skeleton"
    r.lpush(list_key, *map(str, sequence))
    return list_key

def load_category(base_key, category):
    category_key = f"{base_key}/category/{category['id']}"

    if "keypoints" in category:
        category["keypoints"] = load_keypoint_names(category_key, category["keypoints"])
    if "skeleton" in category:
        category["skeleton"] = load_list_of_str(category_key, category["skeleton"])

    r.hset(category_key, mapping=category)
    return category_key

def load_segmentation(base_key, segmentation):
    segmentation_key = f"{base_key}/segmentation"
    if not r.exists(segmentation_key):
        for i, segment in enumerate(segmentation):
            segment_key = f"{segmentation_key}/{i}"
            r.lpush(segment_key, *segment)
            r.rpush(segmentation_key, segment_key)
    return segmentation_key

def load_annotation(base_key, annotation):
    
    annotation_key = f"{base_key}/image/{annotation['image_id']}/annotation/{annotation['id']}"
    
    if not r.exists(annotation_key):
        if "segmentation" in annotation:
            # Replace the 'segmentation' list-of-lists, with a key with a list of keys, that in turn point to the inner lists :)
            annotation["segmentation"] = load_segmentation(annotation_key, annotation["segmentation"])
        
        if "bbox" in annotation:
            # Replace the 'bbox' with a string reepresentaton.load_segmentation
            annotation["bbox"] = str(annotation["bbox"])

        if "keypoints" in annotation:
            annotation["keypoints"] = load_list_of_str(annotation_key, annotation["keypoints"])

        r.hset(annotation_key, mapping=annotation)
    return annotation_key

def load_annotation_jsons_from_zip(zip_file):
    with zipfile.ZipFile(zip_file) as archive:
        for file_name in archive.namelist():
            is_annotation_file = annotation_json_pattern.match(file_name)
            if not is_annotation_file:
                continue
            
            annotation_type = is_annotation_file.group(1)
            dataset_purpose = is_annotation_file.group(2)
            dataset_year = is_annotation_file.group(3)

            if not annotation_type in annotation_types:
                continue
            
            if not dataset_purpose in purpose:
                continue
                
            if not dataset_year in years:
                continue

            with archive.open(file_name) as json_file:
                contents = json.load(json_file)
    
            base_key = f"/dataset/coco/{dataset_year}"
            info_key = f"{base_key}/general/{annotation_type}/{dataset_purpose}"
                
            # info
            if "info" in contents:
                progress.value = f"Loading info for {dataset_purpose} {dataset_year} {annotation_type}"
                load_annotation_info(info_key, contents["info"])

            # licenses
            if "licenses" in contents:
                progress.value = f"Loading licenses for {dataset_purpose} {dataset_year} {annotation_type}"
                for lic in contents["licenses"]:
                    load_license_info(info_key, lic)
                    
            # images
            if "images" in contents:
                progress.value = f"Loading images for {dataset_purpose} {dataset_year} {annotation_type}"
                for image_info in contents["images"]:
                    load_image_info(base_key, image_info)

            # annotations
            if "annotations" in contents:
                progress.value = f"Loading annotations for {dataset_purpose} {dataset_year} {annotation_type}"
                for annotation in contents["annotations"]:
                    load_annotation(base_key, annotation)

            # categories (for "instances" and "person_keypoints")
            if "categories" in contents:
                progress.value = f"Loading categories for {dataset_purpose} {dataset_year} {annotation_type}"
                for category in contents["categories"]:
                    load_category(base_key, category)

            
display(progress)
for archive in annotation_archives:
    progress.value = f"Unzipping {archive}"
    load_annotation_jsons_from_zip(archive)
progress.value = "Done!"


In [ ]:
# cluster.flushall()


# Demos
This is the actual Demo section. Everything above is just preparations.

1. [The Basics](#1.-The-Basics)
2. [Simple Analytics Query](#2.-Simple-Analytics-Query)
3. [Transaction Stream Processing](#3.-Transaction-Stream-Processing)
4. [Custom Command](#4.-Custom-Command)


<a id="demm-basics"></a>
## 1. The Basics
Showcasing some of the basic features and commands of the redgrease package.

In [ ]:
import redgrease
import redgrease.utils

# Create connection / client for single instance Redis
single = redgrease.RedisGears() 

# Create connectoin / client for Redis Cluster 
cluster = redgrease.RedisGears(
    startup_nodes=[
        {"host":"localhost", "port":30001},
        {"host":"localhost", "port":30002},
        {"host":"localhost", "port":30003},
    ]
)


In [ ]:
# The usual Redis commands work as expected
a = single.flushall()
b = single.set("Foo", 21)
c = single.hset("Bar", mapping={"spam":"eggs", "meaning":8})
d = single.hincrby("Bar", "meaning", 34)
e = single.xadd("tlogs::0", {"msg":"START", "from":0, "to":0, "amount":0})

a, b, c, d, e

In [ ]:
# Gears commands can be accessed through the 'gears' attribute

cluster_pystats = cluster.gears.pystats()

print(f"Cluster Redis - Python Stats:\n{cluster_pystats}\n")



cluster_info = cluster.gears.infocluster()

print(f"Cluster Redis - Cluster Info:\n{cluster_info}\n")



cluster_refreshed = cluster.gears.refreshcluster()

print(f"Cluster Redis - Cluster Refresh Response:\n{cluster_refreshed}\n")



single_pystats = single.gears.pystats()

print(f"Single-node Redis - Python Stats:\n{single_pystats}\n")


# Iterate through all Redis key-value records, and return all record data.
all_records_gear = single.gears.pyexecute("GearsBuilder().run()")

print("Single-node Redis - All-records gear: [")
for result in all_records_gear:
    print(f"  {result}")
print("]\n")


# Iterate through all Redis key-value records, and return just the key and type
key_type_gear = single.gears.pyexecute("GearsBuilder().map(lambda record:(record['key'], record['type'])).run()")


print("Single-node Redis - Key-types gear: [")
for result in key_type_gear:
    print(f"  {result}")
print("]\n")


single_record_count = single.gears.pyexecute("GearsBuilder().count().run()")
print(f"Single-node Redis - Record count: {single_record_count}")

### Instead of python functions as strings
RedGrease allows for cunstruction of GearFuntion objects

In [ ]:
record_count = redgrease.KeysOnlyReader().count().run()

cluster_record_count = cluster.gears.pyexecute(record_count)

print(f"Cluster Redis - Total records: {cluster_record_count}")

In [ ]:

# GearFunctions lets you compose and reuse partial functions
images = redgrease.KeysReader("/dataset/coco/*/image/*/info").map(lambda record: record['value'])

image_count = images.count()

square_images = images.filter(lambda img: img['height'] == img['width'])

some_image_urls = square_images.collect().limit(4).map(lambda record: record['coco_url'])

def instance_annotations(year="*"):
    return (
        redgrease.KeysReader(f"/dataset/coco/{year}/image/*/annotation/*")
        .filter(lambda record: record['type'] == "hash")
        .map(lambda record: record["value"])
    )

# Different ways of running
img_cnt = image_count.run(on=cluster)
img_portrait = square_images.run().on(cluster)

annotation_cnt = cluster.gears.pyexecute(instance_annotations().count().run())
img_urls = cluster.gears.pyexecute(some_image_urls)


print(f"Total number of images: {img_cnt}\n")
print(f"Total number of annotations: {annotation_cnt}\n")

from IPython.display import Image
from IPython.core.display import HTML 

print(f"Some square images")
for img_url in img_urls:
    display(Image(url=img_url))





<a id="demo-query"></a>
## 2. Simple Analytics Query 

In [ ]:
def dict_merge(d1, d2):
    return {**d1, **d2}

get_category_ids = (
    redgrease.KeysReader("/dataset/*/category/*")
    .map(lambda record: {record['value']['name']:record['value']['id']})
    .aggregate({},dict_merge, dict_merge)
)

get_category_names = (
    redgrease.KeysReader("/dataset/*/category/*")
    .map(lambda record: {record['value']['id']:record['value']['name']})
    .aggregate({},dict_merge, dict_merge)
)
                         
category_id_lookup = get_category_ids.run(on=cluster)
category_name_lookup = get_category_names.run(on=cluster)
category_id_lookup, len(category_id_lookup)

In [ ]:


def accumulate_categories(image_id, accumulator, annotation):    
    annotation_category_id = annotation.get('category_id',-1)
    accumulator[annotation_category_id] = accumulator.get(annotation_category_id, 0) + 1
    
    return accumulator


def accumulate_category_counts(image_id, accumulator, category_count):
    for category, count in category_count.items():
        accumulator[category] = accumulator.get(category, 0) + count
    return accumulator


def format_img_stats(img_stats):
    return {
        'image_id': img_stats['key'],
        'instances': img_stats['value']
    }

category_count_by_image = instance_annotations(2017).aggregateby(
    extractor = lambda annotation : annotation.get('image_id', -1),
    zero = {},
    seqOp = accumulate_categories,
    combOp = accumulate_category_counts
).map(format_img_stats)

c = category_count_by_image.limit(10).run(on=cluster)
len(c), c, c.errors






In [ ]:
def contstrain(constraints):
    # return a predicate for instance counts where the count of a set of categories
    # Constraints is a dict from category name to a tuple of (min_count, max_count)
    
    id_constraints = { category_id_lookup[cat_name]:x for cat_name, x in constraints.items()}
    
    def predicate(record):
        instances = record['instances']
        
        for cat_id, constraint in id_constraints.items():
            min_count, max_count = constraint
            
            if cat_id not in instances:
                if min_count is not ... and min_count > 0:
                    return False
                continue
            
            if min_count is not ... and instances[cat_id] < min_count:
                return False
            
            if max_count is not ... and instances[cat_id] > max_count:
                return False
            
        return True

    return predicate
    
query_params = {
    'banana': [5, ...],
    'person': [1, 1],
    'orange': [..., 0],
    'truck': [1, 2],
}


query = category_count_by_image.filter(contstrain(query_params))

query_result = query.limit(30).run(on=cluster)

query_result, query_result.errors



In [ ]:
query.count().run(on=cluster)

In [ ]:

image_ids = [result['image_id'] for result in query_result]

image_urls = (
    redgrease.KeysReader("/dataset/coco/2017/image/*/info")
    .map(lambda record: record['value'])
    .filter(lambda img: img['id'] in image_ids)
    .map(lambda img: img["coco_url"])
    .run(on=cluster)
)

for image_url in image_urls:
    display(Image(url=image_url))


<a id="demo-stream"></a>
## 3. Transaction Stream Processing

In [965]:
import random

entity_count = 5
min_start_balance = 100
max_start_balance = 1000


# Create some accounts
for entity_id in range(entity_count):
    start_balance = random.randint(min_start_balance, max_start_balance)
    single.hset(
        f"/entity/{entity_id}",
        mapping={
            "id": entity_id, 
            "balance": start_balance,
            "start_balance": start_balance,
        }
    )
    
    
def attempt_random_transaction(channel, max_amount=100, message="This is a random transaction",):
    single.xadd(
        f">>transactions:{channel}", 
        {
            "msg": message, 
            "from": random.randint(0, entity_count-1), 
            "to": random.randint(0, entity_count-1), 
            "amount": random.randint(1, max_amount),
        }
    )

    
def balance_sheet():
    sum_balance = 0
    for entity_id in range(entity_count):
        current_balance, start_balance = map(int, single.hmget(f"/entity/{entity_id}", "balance", "start_balance"))
        print(f"Entity {entity_id} balance: {current_balance}  ({current_balance-start_balance})")
        sum_balance += current_balance
    print("----------------------------")
    print(f"Total balance   : {sum_balance}")
    return sum_balance

start_total_balance = balance_sheet()

Entity 0 balance: 380  (0)
Entity 1 balance: 497  (0)
Entity 2 balance: 794  (0)
Entity 3 balance: 769  (0)
Entity 4 balance: 837  (0)
----------------------------
Total balance   : 3277


In [967]:

# Transform a key-space event to a transaction
def to_transaction(event):
    transaction = event['value']
    transaction['channel'] = event['key']
    transaction['id'] = event['id']
    return transaction
    
# Register the tranaction on its own unique key.
def register_transaction(transaction):
    transaction['status'] = "pending"
    redgrease.cmd.hset(
        f"/status/{transaction['channel']}/{transaction['id']}",
        mapping=transaction
    )

    
# Handle the transaction safely
def handle_transaction(transaction):
    
    # Log the transaction event to the Redis engine log
    redgrease.log(f"Procesing transaction {transaction['id']}: {transaction}")
    
    # Perform a sequence of commands atomically
    with redgrease.atomic():
        
        # Check if the 'sender' has sufficient balance
        sender_balance = redgrease.cmd.hget(
                f"/entity/{transaction['from']}",
                "balance"
            )
        amount = int(transaction.get('amount', 0))
        
        if not sender_balance or amount > int(sender_balance):
            # If balance is not sufficient, the transaction is marked as failed.
            transaction['status'] = f"FAILED: Missing {int(sender_balance)-amount}"
            
        else:                      
            # If there is sufficient balance, 
            # remove the amount from sender and add it to the recipient
            # and mark as successful
            redgrease.cmd.hincrby(
                f"/entity/{transaction['from']}",
                "balance",
                -amount
            )
            redgrease.cmd.hincrby(
                f"/entity/{transaction['to']}",
                "balance",
                amount
            )
            transaction['status'] = "successful"
        
    # Update the transaction status
    redgrease.cmd.hset(
        f"/status/{transaction['channel']}/{transaction['id']}",
        "status",
        transaction['status']
    )
    redgrease.log(f"Done processing transaction {transaction['id']}: {transaction['status']}")
    return transaction

def handle_unsuccessful_transaction(transaction):
    redgrease.log(f"Handling rejected transaction {transaction['id']}: {transaction}")
    redgrease.cmd.xadd(">>rejected", transaction)

    
import uuid
def dump_to(key):
    def dump(x):
        redgrease.cmd.hset(f"{key}::{uuid.uuid4()}", mapping=dict(x))
    return dump
    
    
    
# Transaction processing pipeline
transsaction_pipe = (
    redgrease.StreamReader()  # Listen to streams
    .map(to_transaction)  # Map events to a 'transaction' dict
    .foreach(register_transaction)  # Register the transactions as 'pending'
    .map(handle_transaction)  # Execute the transaction
    .filter(lambda transaction: transaction['status'] != "successful") # Get the transactions that didn't succeed
    .foreach(handle_unsuccessful_transaction) # Pretend to handle "appropriately"
    .register(prefix=">>transactions:*", batch=10, duration=30) # Listen to transaction stream and use batching
)


# Register the processing pipeline
transsaction_pipe.on(single)


ExecutionResult[bool](True)

In [968]:
for registration in single.gears.dumpregistrations():
    print(f"Registered Gear function {registration.id} has been triggered {registration.RegistrationData.numTriggered} times.")

Registered Gear function 0000000000000000000000000000000000000000-58 has been triggered 0 times.


In [975]:
attempt_random_transaction("sample")

In [977]:
balance_sheet()

Entity 0 balance: 337  (-43)
Entity 1 balance: 497  (0)
Entity 2 balance: 837  (43)
Entity 3 balance: 769  (0)
Entity 4 balance: 837  (0)
----------------------------
Total balance   : 3277


3277

In [978]:
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

# Run a bunch of transactions in parallell
parallell_transaction_jobs = 100
sequential_transactions_count = 100
max_transaction_amount = 500 

def sequential_transactions(channel="foo"):
    def attempt_transactions():
        for transaction_id in range(sequential_transactions_count):
            attempt_random_transaction(
                channel, 
                max_amount=max_transaction_amount,
                message=f"This is a transaction #{transaction_id} on channel {channel}",
            )
    return attempt_transactions
    
def run_in_parallell(jobs):
    with ThreadPoolExecutor() as worker:
        tasks = [worker.submit(job) for job in jobs]

jobs = [ sequential_transactions(nm) for nm in range(parallell_transaction_jobs)]
        
run_in_parallell(jobs)
        

In [979]:
end_total_balance = balance_sheet()
print(f"Total difference: {start_total_balance - end_total_balance}")
print()
for registration in single.gears.dumpregistrations():
    print(f"Registered Gear function {registration.id} has been triggered {registration.RegistrationData.numTriggered} times.")

Entity 0 balance: 78  (-302)
Entity 1 balance: 118  (-379)
Entity 2 balance: 145  (-649)
Entity 3 balance: 2062  (1293)
Entity 4 balance: 874  (37)
----------------------------
Total balance   : 3277
Total difference: 0

Registered Gear function 0000000000000000000000000000000000000000-58 has been triggered 1793 times.


In [983]:
# Show errors
for exe in single.gears.dumpexecutions():
    batch = single.gears.getresults(exe.executionId)
    if batch.errors: 
        print(f"Errors: {batch.errors}")
        
    if batch:
        for remaining_transactions in batch.value:
            print(f"Unsuccessful transaction: {remaining_transactions}")


Unsuccessful transaction: b"{'to': '2', 'from': '2', 'msg': 'This is a transaction #83 on channel 97', 'amount': '374', 'channel': '>>transactions:97', 'id': '1616675582825-1', 'status': 'FAILED: Missing -276'}"
Unsuccessful transaction: b"{'to': '2', 'from': '1', 'msg': 'This is a transaction #86 on channel 97', 'amount': '496', 'channel': '>>transactions:97', 'id': '1616675582831-1', 'status': 'FAILED: Missing -5'}"
Unsuccessful transaction: b"{'to': '1', 'from': '1', 'msg': 'This is a transaction #73 on channel 99', 'amount': '374', 'channel': '>>transactions:99', 'id': '1616675582809-0', 'status': 'FAILED: Missing -360'}"
Unsuccessful transaction: b"{'to': '1', 'from': '1', 'msg': 'This is a transaction #75 on channel 99', 'amount': '455', 'channel': '>>transactions:99', 'id': '1616675582811-0', 'status': 'FAILED: Missing -441'}"
Unsuccessful transaction: b"{'to': '4', 'from': '1', 'msg': 'This is a transaction #77 on channel 99', 'amount': '381', 'channel': '>>transactions:99', 'i

### Reset Transaction Stream Processing Demo

In [ ]:
# Unregister all registrations
for reg in single.gears.dumpregistrations():
    single.gears.unregister(reg.id)

# Remove all executions
for exe in single.gears.dumpexecutions():
    signle.gears.dro
    single.gears.dropexecution(str(exe.executionId))

# Clear all keys
single.flushall()

# Check that there are no keys
single.keys()


<a id="demo-command"></a>
## 4. Custom Command

In [ ]:
print("Demo 4 - Custom Command")
cluster = redgrease.RedisGears(
    startup_nodes=[
        {"host":"localhost", "port":30001},
        {"host":"localhost", "port":30002},
        {"host":"localhost", "port":30003},
    ]
)
cluster.hgetall("/dataset/COCO/annotations/category/47")

In [ ]:
cluster.hget("/dataset/coco/2014/image/74827/info", "coco_url")